In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
print(tf.VERSION)
print(tf.keras.__version__)

1.15.0
2.2.4-tf


In [4]:
# #download the model to local so it can be used again and again
# !mkdir module/module_elmo2
# # Download the module, and uncompress it to the destination folder. 
# !curl -L "https://tfhub.dev/google/elmo/2?tf-hub-format=compressed" | tar -zxvC module/module_elmo2

In [5]:
train = pd.read_csv("/Users/mac/Desktop/Thesis2/ELMO/Train_elmo.csv")
test = pd.read_csv("/Users/mac/Desktop/Thesis2/ELMO/Test_elmo.csv")

In [6]:
train.head()

,label,cleand
0,0,grow b 1965 watch loving thunderbird mate scho...
1,0,put movie dvd player sit coke chip expectation...
2,0,people know particular time past like feel ne ...
3,0,even though great interest biblical movie bore...
4,1,-PRON- be die hard dad army fan nothing ever c...


In [7]:
# second round cleaning
# get rid of the ''
train['cleand'] = train['cleand'].str.replace("'"," ")
test['cleand'] = test['cleand'].str.replace("'"," ")
train['cleand'] = train['cleand'].str.replace("-PRON-"," ")
test['cleand'] = test['cleand'].str.replace("-PRON-"," ")
train['cleand'] = train['cleand'].str.replace("  "," ")
test['cleand'] = test['cleand'].str.replace("  "," ")
train['cleand'] = train['cleand'].str.replace("   "," ")
test['cleand'] = test['cleand'].str.replace("   "," ")

In [8]:
train.cleand.iloc[0]

'grow b 1965 watch loving thunderbird mate school watch play thunderbird school lunch school want virgil scott one want alan count 5 become art form take child see movie hoping would get glimpse love child bitterly disappointing high point snappy theme tune could compare original score thunderbird thankfully early saturday morning one television channel still play rerun series gerry anderson wife create jonatha frak hand director chair version completely hopeless waste film utter rubbish cgi remake may acceptable replacing marionette homo sapiens subsp sapiens huge error judgment'

In [9]:
# elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [10]:
# # just a random sentence
# x = ["Roasted ants are a popular snack in Columbia"]
# # Extract ELMo features
# embeddings = elmo(x, signature="default", as_dict=True)["elmo"]
# embeddings.shape

In [11]:
X = np.array(train["cleand"])
y = np.array(train["label"])

array([0, 0, 0, 0])

In [12]:
from collections import Counter
Counter(y)

Counter({0: 5043, 1: 4957})

In [13]:
embed = hub.Module("module/module_elmo2")
def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [14]:
import tensorflow.keras as keras 
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

Using TensorFlow backend.


In [15]:
def build_model(): 
    input_text = Input(shape=(1,), dtype="string")
    embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
    dense = Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(embedding)
    pred = Dense(1, activation='sigmoid')(dense)
    model = Model(inputs=[input_text], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [16]:
model_elmo = build_model()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [17]:
model_elmo.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 262,657
Trainable params: 262,657
Non-trainable params: 0
_________________________________________________________________


In [18]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model_elmo.fit(X, y, epochs=3, batch_size=8, validation_split = 0.2)
    model_elmo.save_weights('./model_elmo_weights.h5')



Train on 8000 samples, validate on 2000 samples
Epoch 1/3
  88/8000 [..............................] - ETA: 3:52:07 - loss: 1.2299 - accuracy: 0.5568

KeyboardInterrupt: 